## 读取csv写入到crm_data，49写入，50读取

In [ ]:
import numpy as np
import pandas as pd
import pymysql

def test_execute_many(conn, sql, l):
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:
        try:
            print(11)
            cursor.executemany(sql, l)
            print("======")
            conn.commit()
        except Exception as e:
            print(e, "222")
            conn.rollback()
        cursor.close()
        conn.close()
        
def update_crm(l):
    
    sql = "update user_information1 set name=%s, nick_name=%s, age=%s, gender=%s, icon_url=%s, customer_type=%s, birthday=%s,\
    certificate_type=%s, certificate_number=%s, province=%s, city=%s, country=%s, township=%s, address=%s, industry=%s,\
    education_level=%s, personal_mothly_income=%s, family_mothly_income=%s, family_status=%s,telephone=%s, remarks=%s where phone=%s;"
    print('开始更新')
    test_execute_many(conn, sql, l)
    print("完成")
    
def reader_csv():
#     data = pd.read_csv('result_1.16.csv', keep_default_na=False)
    data = pd.read_csv('result_2_9.csv', keep_default_na=False)
    data['gender'].replace('男', 1, inplace=True)
    data['gender'].replace('女', 2, inplace=True)
    data['gender'].replace(' ', 0, inplace=True)
    phone = data.phone
    phone = [str(p) for p in phone]
    data = data.drop('phone',axis=1)
    data.insert(21,'phone',phone)
    data = np.array(data)
    l=[]
    for i in data:
        l.append(tuple([None if j=='' else j for j in i]))
    print(l[:1000])
#     return l

if __name__ == '__main__':
    conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
    result = reader_csv()
    update_crm(result)

## 抽取phone到clickhouse

In [ ]:
from clickhouse_driver import Client
import pymysql

def query_phone():
    conn = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
    cur = conn.cursor()
    sql = 'select phone from user_information1;'
    cur.execute(sql)
    phones = cur.fetchone()
    cursor.close()
    conn.close()
    print(phones)

def write_phone(phones):
    client = Client(host='96.2.1.19',port=9000, user='default', password='gW6DO83Q', database='sgmw_big_data')
    sql = ''
    client.execute(sql)
    
if __name__ == '__main__':
    query_phone()

## clickhouse-to-crm

In [ ]:
import numpy as np
import pandas as pd
import pymysql
from clickhouse_driver import Client
import time

def test_execute_many(conn, sql, l):
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:
        try: 
            print(11, l)
            cursor.executemany(sql, l)
            print("======")
            conn.commit()
        except Exception as e:
            print(e, "222")
            conn.rollback()
        cursor.close()
        conn.close()
        
def update_crm(l):
    
#     sql = "update user_information1 set name=%s, nick_name=%s, age=%s, gender=%s, icon_url=%s, customer_type=%s, birthday=%s,\
#     certificate_type=%s, certificate_number=%s, province=%s, city=%s, country=%s, township=%s, address=%s, industry=%s,\
#     education_level=%s, personal_mothly_income=%s, family_mothly_income=%s, family_status=%s,telephone=%s, remarks=%s where phone=%s;"
#     print(l[0], "==")
    
    sql = "insert into crm_data.user_information_detail(phone,name,name_source,age,age_source,gender,gender_source,birthday,birthday_source,\
    certificate_type,certificate_type_source,certificate_number,certificate_number_source,province,province_source,city,city_source,\
    country,country_source,township,township_source,address,address_source,postcode,postcode_source,industry,industry_source,\
    education_level,education_level_source,personal_mothly_income,personal_mothly_income_source,family_mothly_income,\
    family_mothly_income_source,family_status,family_status_source,telephone,telephone_source,created_id) values \
    (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    print('开始更新')
#     print(sql)
    test_execute_many(conn, sql, l)
    print("完成")
    
def query_data():
#     client = Client(host='96.2.0.54',port=9000, user='default', database='sgmw_big_data')
    conn = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8')
    cursor = conn.cursor()
    sql = "SELECT \
    CONTACTPHONE AS phone, \
    NAME AS name,\
    '1' AS name_source,\
    cast(AGE as UNSIGNED INTEGER) as age,\
    '1' AS age_source,\
    (CASE \
        WHEN GENDER = '男' THEN '1'\
        WHEN GENDER = '女' THEN '2'\
        ELSE '0' END) AS gender,\
    '1' AS gender_source,\
    (CASE \
        WHEN BIRTH is null or BIRTH = '________' or BIRTH = '3050000' THEN null \
        WHEN BIRTH is not null THEN BIRTH END) AS birthday,\
    '1' AS birthday_source,\
    IDTYPE AS certificate_type,\
    '1' AS certificate_type_source,\
    IDNUMBER AS certificate_number,\
    '1' AS certificate_number_source,\
    PROVINCE AS province,\
    '1' AS province_source,\
    CITY AS city,\
    '1' AS city_source,\
    DISTRICT AS country,\
    '1' AS country_source,\
    TOWNSHIP AS township,\
    '1' AS township_source,\
    ACTUALADDRESS AS address,\
    '1' AS address_source,\
    ZIPCODE AS postcode,\
    '1' AS postcode_source,\
    INDUSTRY AS industry,\
    '1' AS industry_source,\
    LITERACY AS education_level,\
    '1' AS education_level_source,\
    INCOMELEVELMONTHLY AS personal_mothly_income,\
    '1' AS personal_mothly_income_source,\
    FAMILYLEVELMONTHLY AS family_mothly_income,\
    '1' AS family_mothly_income_source,\
    FAMILYNAME AS family_status,\
    '1' AS family_status_source,\
    SEATNUMBER AS telephone,\
    '1' AS telephone_source,\
    'liuyuntao' AS created_id,\
    max(DATA_TIMESTAMP) \
FROM \
    sgmw_big_data.vactualsale group by CONTACTPHONE limit 10000,20000;"
    cursor.execute(sql)
    query_result = cursor.fetchall()   
    l = []
    for i in query_result:
        l.append(i[:38])
#     print(l)
    return l
    
if __name__ == '__main__':
    start_time = time.time()
    conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
    result = query_data()
    update_crm(result)
    print(time.time() - start_time)

## 车辆表 最新时间：03-12

In [ ]:
import pymysql

conn = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8')
cursor= conn.cursor()
sql= "SELECT \
    '1' AS name_source,\
    NAME AS name,\
    '1' AS age_source,\
    cast(AGE as UNSIGNED INTEGER) as age,\
    '1' AS gender_source,\
    (CASE \
        WHEN GENDER = '男' THEN '1'\
        WHEN GENDER = '女' THEN '2'\
        ELSE '0' END) AS gender,\
    '1' AS birthday_source,\
    (CASE \
        WHEN BIRTH is null or BIRTH = '________' or BIRTH = '3050000' THEN null \
        WHEN BIRTH is not null THEN BIRTH END) AS birthday,\
    '1' AS certificate_type_source,\
    IDTYPE AS certificate_type,\
    '1' AS certificate_number_source,\
    IDNUMBER AS certificate_number,\
    PROVINCE AS province,\
    '1' AS province_source,\
    CITY AS city,\
    '1' AS city_source,\
    DISTRICT AS country,\
    '1' AS country_source,\
    TOWNSHIP AS township,\
    '1' AS township_source,\
    ACTUALADDRESS AS address,\
    '1' AS address_source,\
    ZIPCODE AS postcode,\
    '1' AS postcode_source,\
    INDUSTRY AS industry,\
    '1' AS industry_source,\
    LITERACY AS education_level,\
    '1' AS education_level_source,\
    INCOMELEVELMONTHLY AS personal_mothly_income,\
    '1' AS personal_mothly_income_source,\
    FAMILYLEVELMONTHLY AS family_mothly_income,\
    '1' AS family_mothly_income_source,\
    FAMILYNAME AS family_status,\
    '1' AS family_status_source,\
    '1' AS telephone_source,\
    SEATNUMBER AS telephone,\
    CONTACTPHONE AS phone, \
    max(DATA_TIMESTAMP) \
FROM \
    sgmw_big_data.vactualsale where CONTACTPHONE is not null and `DATA_TIMESTAMP` between '2020-03-10 00:00:00' and '2020-03-12 00:00:00' group by CONTACTPHONE ";
cursor.execute(sql)
va_result = cursor.fetchall()
cursor.close()
conn.close()
va = []
for i in va_result:
    va.append(i[:37])

In [ ]:
len(va)

In [ ]:
print(va[:3])

In [ ]:
import time
import pymysql

db = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cur = db.cursor()
sql = "select phone from crm_data.user_information_detail where phone is not null and phone != '';"
cur.execute(sql)
phones = cur.fetchall()
total_phone = []
for ph in phones:
    total_phone.append(ph[0])
cur.close()
db.close()

In [ ]:
len(total_phone)

In [ ]:
yes_phone = []
no_phone = []

for i in va:
    if str(i[-1]) in total_phone:
        yes_phone.append(i)                             
    else:
        no_phone.append(i)

In [ ]:
len(yes_phone), len(no_phone)

In [ ]:
print(yes_phone[:4])
print(no_phone[:4])

In [ ]:
for i in yes_phone:
    if len(i[3:4]) > 11:
        print(i)

In [ ]:
conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cursor = conn.cursor()

try:
    sql = "update crm_data.user_information_detail set name_source=(CASE WHEN name is not null or name != '' THEN name_source ELSE %s END), \
    name=(CASE WHEN name is not null or name != '' THEN name ELSE %s END), \
    age_source=(CASE WHEN age is not null or age != '' THEN age_source ELSE %s END), \
    age=(CASE WHEN age is not null or age != '' THEN age ELSE %s END), \
    gender_source=(CASE WHEN gender is not null or gender != '' THEN gender_source ELSE %s END), \
    gender=(CASE WHEN gender is not null or gender != '' THEN gender ELSE %s END), \
    birthday_source=(CASE WHEN birthday is not null or birthday != '' THEN birthday_source ELSE %s END), \
    birthday=(CASE WHEN birthday is not null or birthday != '' THEN birthday ELSE %s END), \
    certificate_type_source=(CASE WHEN certificate_type is not null or certificate_type != '' THEN certificate_type_source ELSE %s END), \
    certificate_type=(CASE WHEN certificate_type is not null or certificate_type != '' THEN certificate_type ELSE %s END), \
    certificate_number_source=(CASE WHEN certificate_number is not null or certificate_number !='' THEN certificate_number_source ELSE %s END),\
    certificate_number=(CASE WHEN certificate_number is not null or certificate_number != '' THEN certificate_number ELSE %s END),\
    province=%s,province_source=%s,city=%s,city_source=%s,country=%s,country_source=%s,township=%s,township_source=%s,\
    address=%s,address_source=%s,postcode=%s,postcode_source=%s,industry=%s,industry_source=%s,\
    education_level=%s,education_level_source=%s,personal_mothly_income=%s,personal_mothly_income_source=%s,family_mothly_income=%s,\
    family_mothly_income_source=%s,family_status=%s,family_status_source=%s,\
    telephone_source=(CASE WHEN telephone is not null or telephone != '' THEN telephone_source ELSE %s END), \
    telephone=(CASE WHEN telephone is not null or telephone != '' THEN telephone ELSE %s END), \
    updated_id='liuyuntao' where phone=%s;"
    print("开始更新")
    cursor.executemany(sql, yes_phone)
    conn.commit()
    print("更新结束")
except Exception as e:
    print(e, "出错")
#     conn.rollback()

In [ ]:
try:
    sql1 = "insert into crm_data.user_information_detail(name_source,name,age_source,age,gender_source,gender,birthday_source,birthday,\
    certificate_type_source,certificate_type,certificate_number_source,certificate_number,province,province_source,city,city_source,\
    country,country_source,township,township_source,address,address_source,postcode,postcode_source,industry,industry_source,\
    education_level,education_level_source,personal_mothly_income,personal_mothly_income_source,family_mothly_income,\
    family_mothly_income_source,family_status,family_status_source,telephone_source,telephone,created_id,phone) values \
    (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,'liuyuntao',%s);"
    print('开始插入')
    cursor.executemany(sql1, no_phone)
    conn.commit()
    print("插入完成")
except Exception as e:
    print(e, "出错")
    conn.rollback()
cursor.close()
conn.close()

## 试驾数据更新crm

In [ ]:
import time
import pymysql

# def execute_many(conn, sql, driver_result):
#     with conn.cursor(pymysql.cursors.DictCursor) as cursor:
#         try:
#             print(11)
#             cursor.executemany(sql, driver_result)
#             print("======")
#             conn.commit()
#         except:
#             pass
#         continue
# #             conn.rollback()


def update_crm(conn, cursor, l1, l):
    for i in l1:
        j = 1
        if str(i[-1]) in l:
            print(i)
            sql = "update crm_data.user_information_detail set name_source=(CASE WHEN name is not null or name != '' THEN name_source ELSE %s END), \
            name=(CASE WHEN name is not null or name != '' THEN name ELSE %s END), \
            certificate_number_source=(CASE WHEN certificate_number is not null or certificate_number !='' THEN certificate_number_source ELSE %s END),\
            certificate_number=(CASE WHEN certificate_number is not null or certificate_number != '' THEN certificate_number ELSE %s END),\
            updated_id='liuyuntao' where phone=%s;"
            print("开始更新")
            cursor.executemany(sql, i)
            conn.commit()
            print("更新结束", j)
        else:
            print(i)
            sql = "insert into crm_data.user_information_detail(name_source, name, certificate_number_source, certificate_number, phone, \
            created_id) value (%s, %s, %s, %s, %s, 'liuyuntao');"
            print("开始插入")
            print(sql)
            cursor.executemany(sql, i)
            conn.commit()
            print("插入结束", j)
        j += 1

    
def query_data():
    db = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8')
    cur = db.cursor()
    sql = "select 3 as name_source, dd.CUSTOMER_NAME as name, 3 as certificate_number_source, dd.PROTOCOL_CARD as certificate_number,dd.`MOBILE` as phone, max(dd.`DATA_TIMESTAMP` ) from sgmw_big_data.driver_data dd where dd.`DATA_TIMESTAMP` <'2020-02-13 00:00:00' group by dd.`MOBILE`;"
    cur.execute(sql)
    driver_result = cur.fetchall()
    cur.close()
    db.close()
    l1 = []
    for ds in driver_result:
        l1.append(ds[:5])
    return l1

def get_total_phone():
    db = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
    cur = db.cursor()
    sql = "select phone from crm_data.user_information_detail;"
    cur.execute(sql)
    total_phone = cur.fetchall()
    l = []
    for ph in total_phone:
        l.append(ph[0])
    return l

if __name__ == '__main__':
    start_time = time.time()
    conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
    cursor = conn.cursor()
    driver_result = query_data()
    total_phone = get_total_phone()   
    update_crm(conn, cursor, driver_result, total_phone)
    cursor.close()
    conn.close()
    print(time.time() - start_time)

In [ ]:
import time
import pymysql

db = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cur = db.cursor()
sql = "select phone from crm_data.user_information_detail where phone is not null and phone != '';"
cur.execute(sql)
driver_phones = cur.fetchall()
driver_total_phone = []
for dr in driver_phones:
    driver_total_phone.append(dr[0])
cur.close()
db.close()

In [ ]:
len(driver_total_phone)

In [ ]:
conn = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8')
cursor= conn.cursor()
sql = "select 3 as name_source, dd.CUSTOMER_NAME as name, 3 as certificate_number_source, dd.PROTOCOL_CARD as certificate_number,\
dd.`MOBILE` as phone, max(dd.`DATA_TIMESTAMP`) from sgmw_big_data.driver_data dd \
where dd.MOBILE is not null and dd.`DATA_TIMESTAMP` between '2020-03-10 00:00:00' and '2020-03-12 00:00:00' group by dd.`MOBILE`;"
cursor.execute(sql)
driver_result = cursor.fetchall()
cursor.close()
conn.close()
driver_phones = []
for ds in driver_result:
    driver_phones.append(ds[:5])

In [ ]:
# len(driver_total_phone)
len(driver_phones)

In [ ]:
print(driver_phones[:3])

In [ ]:
yes_phone1 = []
no_phone1 = []

for i in driver_phones:
    if str(i[-1]) in driver_total_phone:
        yes_phone1.append(i)
    else:
        no_phone1.append(i)

In [ ]:
len(yes_phone1), len(no_phone1)

In [ ]:
yes_phone1

In [ ]:
no_phone1

In [ ]:
conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cursor = conn.cursor()
try:
    sql = "update crm_data.user_information_detail set name_source=(CASE WHEN name is not null or name != '' THEN name_source ELSE %s END), \
    name=(CASE WHEN name is not null or name != '' THEN name ELSE %s END), \
    certificate_number_source=(CASE WHEN certificate_number is not null or certificate_number !='' THEN certificate_number_source ELSE %s END),\
    certificate_number=(CASE WHEN certificate_number is not null or certificate_number != '' THEN certificate_number ELSE %s END),\
    updated_id='liuyuntao' where phone=%s;"
    print("开始更新")
    cursor.executemany(sql, yes_phone1)
    conn.commit()
    print("更新结束")
except Exception as e:
    print(e)

In [ ]:
try:
    sql = "insert into crm_data.user_information_detail(name_source, name, certificate_number_source, certificate_number, phone, \
    created_id) values (%s, %s, %s, %s, %s, 'liuyuntao');"
    print("开始插入")
    cursor.executemany(sql, no_phone1)
    conn.commit()
    print("插入结束")
except Exception as e:
    print(e)
cursor.close()
conn.close()

## 维修

In [ ]:
import time
import pymysql

db = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cur = db.cursor()
sql = "select phone from crm_data.user_information_detail where phone is not null and phone != '';"
cur.execute(sql)
repair_phones = cur.fetchall()
repair_total_phone = []
for re in repair_phones:
    repair_total_phone.append(re[0])
cur.close()
db.close()

In [ ]:
# from clickhouse_driver import Client

In [ ]:
# client = Client(host='96.2.0.54',port=9000, user='default', database='sgmw_big_data')
# sql = "select '4' as name_source, arrayElement(groupArray(OWNER_NAME),-1) as name, '4' as certificate_type_source, arrayElement(groupArray(CERTIFICATE_TYPE), -1) as certificate_type, '4' as certificate_number_source, arrayElement(groupArray(CERTIFICATE_NO), -1) as certificate_number, '4' as telephone_source, arrayElement(groupArray(FAMILY_TEL), -1) as telephone, left(`OWNER_TEL`, 11) as phone from sgmw_big_data.REPAIR_INFO ri where ri.`DATA_TIMESTAMP` between '2020-02-16 00:00:00' and '2020-02-19 00:00:00' GROUP BY ri.OWNER_TEL;"
# result = client.execute(sql)

In [ ]:
len(repair_total_phone)

In [ ]:
import pymysql

conn = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8',connect_timeout=120)
cursor= conn.cursor()
sql= "select '4' as name_source, OWNER_NAME as name, '4' as certificate_type_source, `CERTIFICATE_TYPE` as certificate_type, \
'4' as certificate_number_source, CERTIFICATE_NO as certificate_number, '4' as telephone_source, FAMILY_TEL as telephone, \
left(`OWNER_TEL`, 11) as phone from sgmw_big_data.repair_info ri where ri.OWNER_TEL is not null and ri.`DATA_TIMESTAMP` \
between '2020-03-09 00:00:00' and '2020-03-10 00:00:00' group by ri.OWNER_TEL;";
cursor.execute(sql)
repair_result = cursor.fetchall()
cursor.close()
conn.close()

In [ ]:
len(repair_result)

In [ ]:
# import pymysql
# import pandas as pd

# conn = pymysql.connect(host='172.22.14.115', port=8098, user='root', password='system',database='lyt',charset='utf8')
# cursor= conn.cursor()
# sql= "SELECT rd.name_source,rd.name,rd.certificate_type_source,rd.certificate_type,rd.certificate_number_source ,rd.certificate_number,rd.telephone_source, rd.telephone, 'liuyuntao' as 'updated_id',rd.phone FROM lyt.repair_data rd WHERE rd.phone in (SELECT tp.phone FROM lyt.total_phone tp group by tp.phone);"
# cursor.execute(sql)
# yes_phone = cursor.fetchall()

In [ ]:
# len(yes_phone)

In [ ]:
# l=[]
# for i in yes_phone:
#     l.append(tuple([None if j=='' else j for j in i]))

In [ ]:
# l[:10]

In [ ]:
import time

conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cursor = conn.cursor()
start_time = time.time()
n = 1000000
for j in [l[i:i + n] for i in range(4000000, len(l), n)]:
    try:
        sql = "update crm_data.user_information_detail set \
        name_source=(CASE WHEN name is not null or name != '' THEN name_source ELSE %s END), \
        name=(CASE WHEN name is not null or name != '' THEN name ELSE %s END), \
        certificate_type_source=(CASE WHEN certificate_type is not null or certificate_type !='' THEN certificate_type_source ELSE %s END),\
        certificate_type=(CASE WHEN certificate_type is not null or certificate_type != '' THEN certificate_type ELSE %s END),\
        certificate_number_source=(CASE WHEN certificate_number is not null or certificate_number != '' THEN certificate_number_source ELSE %s END), \
        certificate_number=(CASE WHEN certificate_number is not null or certificate_number != '' THEN certificate_number ELSE %s END), \
        telephone_source=(CASE WHEN telephone is not null or telephone != '' THEN telephone_source ELSE %s END), \
        telephone=(CASE WHEN telephone is not null or telephone != '' THEN telephone ELSE %s END), \
        updated_id=%s where phone=%s;"
        print("开始更新")
        cursor.executemany(sql, j)
        conn.commit()
        print("更新结束")
    except Exception as e:
        print(e, "出错")
        conn.rollback()
cursor.close()
conn.close()
print(time.time() - start_time)

## 精品

In [ ]:
import pymysql

conn = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8')
cursor= conn.cursor()
sql= "select '5' as gender_source, `SEX` as gender, '5' as birthday_source, `BIRTHDAY` as birthday, `USERPHONE`, \
max(`DATA_TIMESTAMP`) as phone from sgmw_big_data.products_order po where po.`DATA_TIMESTAMP` \
between '2020-03-09 00:00:00' and '2020-03-10 00:00:00' group by `USERPHONE`;";
cursor.execute(sql)
products_order_result = cursor.fetchall()
cursor.close()
conn.close()

In [ ]:
len(products_order_result)

In [ ]:
 l = [i[:5] for i in products_order_result]

In [ ]:
l

In [ ]:
import time
import pymysql

db = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cur = db.cursor()
sql = "select phone from crm_data.user_information_detail where phone is not null and phone != '';"
cur.execute(sql)
total_phone = cur.fetchall()
l1 = []
for ph in total_phone:
    l1.append(ph[0])
cur.close()
db.close()

In [ ]:
len(l1)

In [ ]:
yes_phone = []
no_phone = []

for i in l:
    if str(i[-1]) in l1:
        yes_phone.append(i)
    else:
        no_phone.append(i)

In [ ]:
len(yes_phone), len(no_phone)

In [ ]:
# 删除异常生日
for m in yes_phone:
    if len(str(m[3:4][0])) >10:
        yes_phone.remove(m)

In [ ]:
len(yes_phone)

In [ ]:
conn = pymysql.connect(host='96.4.1.49', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cursor = conn.cursor()

try:
    sql = "update crm_data.user_information_detail set gender_source=(CASE WHEN gender is not null or gender != '' THEN gender_source ELSE %s END), \
    gender=(CASE WHEN gender is not null or gender != '' THEN gender ELSE %s END), \
    birthday_source=(CASE WHEN birthday is not null or birthday !='' THEN birthday_source ELSE %s END),\
    birthday=(CASE WHEN birthday is not null or birthday != '' THEN birthday ELSE %s END),\
    updated_id='liuyuntao' where phone=%s;"
    print("开始更新")
    cursor.executemany(sql, yes_phone)
    conn.commit()
    print("更新结束")
except Exception as e:
    print(e)
    conn.rollback()

In [ ]:
try:
    sql = "insert into crm_data.user_information_detail(gender_source, gender, birthday_source, birthday, phone, \
    created_id) values (%s, %s, %s, %s, %s, 'liuyuntao');"
    print("开始插入")
    cursor.executemany(sql, no_phone)
    conn.commit()
    print("插入完成")
except Exception as e:
    print(e)
    conn.rollback()
cursor.close()
conn.close()

In [ ]:
from clickhouse_driver import Client

client = Client(host='96.2.0.54',port=9000, user='default', database='sgmw_big_data')
sql = "SELECT ri2.DEALER_CODE as dealercode,'4' as name_source, ri2.OWNER_NAME as name, '4' as certificate_type_source, ri2.CERTIFICATE_TYPE as certificate_type, '4' as certificate_number_source, ri2.CERTIFICATE_NO as certificate_number, '4' as telephone_source, ri2.FAMILY_TEL as telephone, left(ri2.OWNER_TEL, 11) as phone FROM (SELECT OWNER_TEL, CERTIFICATE_NO FROM sgmw_big_data.REPAIR_INFO WHERE DATA_TIMESTAMP <= '2020-03-13 00:00:00' GROUP BY OWNER_TEL, CERTIFICATE_NO) ri JOIN sgmw_big_data.REPAIR_INFO ri2 ON ri.OWNER_TEL = ri2.OWNER_TEL AND ri.CERTIFICATE_NO = ri2.CERTIFICATE_NO WHERE ri.`DATA_TIMESTAMP` <= '2020-03-13 00:00:00';"
data = client.execute(sql)

In [1]:
import time
import pymysql

db = pymysql.connect(host='96.4.1.50', port=3306, user='crmroot', password='$lS1#te$3rvzD',database='crm_data',charset='utf8')
cur = db.cursor()
sql = "select certificate_number,phone from crm_data.user_information_detail where phone is not null and phone != '' and certificate_number is not null and certificate_number != '';"
cur.execute(sql)
driver_result = cur.fetchall()
cur.close()
db.close()

In [2]:
driver_result

(('440921196706148016', '"020-32703265/1'),
 ('131081197011091041&n', '&nbsp;132906641'),
 ('510104197705234863', '(028)65865826'),
 ('511130196505161419', '(028)66485183'),
 ('510321198208282452', '(028)66872716'),
 ('513027197812025911', '(028)86862328'),
 ('510102195804121272', '(028)89844287'),
 ('511026196705113710', '(028)89922607'),
 ('610124196503212716', '(029)85188593'),
 ('610126197308032135', '(029)86052559'),
 ('755387696000000000', '(051)5567662'),
 ('52010119660805341X', '(0581)6602290'),
 ('522724196809274138', '(0584)2384663'),
 ('795266910000000000', '(0584)6226567'),
 ('441203198501230016', '(0758)2692803'),
 ('440924620917365', '(0758)3834726'),
 ('442824196810152010', '(0758)5792315'),
 ('527500', '(0758)6577651'),
 ('7491770940000000', '(0758)7882368'),
 ('7491770940000000', '(0758)7882368'),
 ('707623236000000000', '(0766)8211037'),
 ('512225196703156683', '(0812)5168972'),
 ('756632366000000000', '(0812)6622700'),
 ('510921196805282358', '(0812)6838275'),
 ('512

In [3]:
conn = pymysql.connect(host='96.2.0.53', port=3306, user='root', password='Sgmw5050@',database='sgmw_big_data',charset='utf8')
cursor= conn.cursor()
sql = "select DEALER_CODE as dealercode,3 as name_source, CUSTOMER_NAME as name, 3 as certificate_number_source, (CASE WHEN PROTOCOL_CARD is not NULL AND PROTOCOL_CARD != '' THEN PROTOCOL_CARD ELSE NULL END) as certificate_number, MOBILE as phone, max(DATA_TIMESTAMP) from sgmw_big_data.driver_data where DATA_TIMESTAMP <= '2020-03-13 00:00:00' group by phone, certificate_number;"
cursor.execute(sql)
total_result = cursor.fetchall()
cursor.close()
conn.close()

In [4]:
l = []
for ds in total_result:
    l.append(ds[:6])

In [5]:
l

[('B11000203', 3, '侯勇', 3, 'null', '13001065308'),
 ('B11000201', 3, '王岳', 3, '110106198208032112', '13001096540'),
 ('B110002', 3, '汤德全', 3, '110105195405263000', '13001102969'),
 ('B120003', 3, '杨洪军', 3, '120111197803023000', '13001309052'),
 ('B120003', 3, '杨洪军', 3, '201111178000235186', '13001309052'),
 ('B120003', 3, '李家朋', 3, '411421199104244000', '13001327672'),
 ('B13002005', 3, '司维娜', 3, '130203198208101229', '13001410516'),
 ('B370008', 3, '张善彬', 3, '370122196305035000', '13001455561'),
 ('B130003', 3, '张红山', 3, '130432198112281000', '13001495732'),
 ('B370004', 3, '魏玉胜', 3, '370303196504261000', '13001506165'),
 ('B37000302', 3, '王东', 3, '372330197110290000', '13001521229'),
 ('B370003', 3, '杨智强', 3, '532522199103152000', '13001524119'),
 ('9370062', 3, '张艳涛', 3, '372328197806248000', '13001525688'),
 ('B37001201', 3, '玄玉莲', 3, '370783197409307000', '13001535079'),
 ('B370012', 3, '高鹏', 3, '107841198001160000', '13001550525'),
 ('B370012', 3, '王维芹', 3, '372923197010272000', 

In [6]:
yes_phone = []
no_phone = []

for i in l:
    if i[-2:] in driver_result:
        yes_phone.append(i)
    else:
        no_phone.append(i)

KeyboardInterrupt: 

In [8]:
len(yes_phone),len(no_phone)

(253, 2680)

In [1]:
l = ('945008826', '1', '郭晓霞', '1', None, '1', '2', '1', None, '1', '身份证', '1', '450204199503040620', '12345678901', '广西壮族自治区', '1', '柳州市', '1', None, '1', None, '1', None, '1', '545007', '1', '其他', '1', '高中/中专/技校', '1', '3001-5000元', '1', '3001-5000元', '1', '单身', '1', '1', '')
print(l[12:14])


('450204199503040620', '12345678901')
